In [5]:
import cv2
import os
import re
from prettyprinter import pprint
from PIL import Image, ImageEnhance

data = []

def preprocess(image):
    img = cv2.imread(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cv2.imwrite(image, img)
    return image

def get_box_file(image, output_path):
    command = 'tesseract {} {} batch.nochop makebox'.format(image, output_path)
    os.system(command)

def get_parsed_output(image, output_file):
    command = 'tesseract {} {} -l eng --psm 4'.format(image, output_file)
    os.system(command)

def store_separate_lines(box_file):
    '''
    Method to separate lines along with box cooridnates
    into nested list.
    '''

    text = ''
    with open(box_file, 'r', encoding='utf-8') as fp:
        text = fp.read()

    text = text.splitlines()

    ch_coords = []

    for ch in text:
        ch_split = ch.split()
        ch_coords.append(ch_split)

    lines = []
    line = []
    max_diff = [1]

    for i in range(len(ch_coords)-1):
        c = ch_coords[i][0]

        if int(ch_coords[i][1]) <= int(ch_coords[i+1][1]) or int(ch_coords[i][1]) - int(ch_coords[i+1][1]) in max_diff:
            line.append(ch_coords[i])
        else:
            line.append(ch_coords[i])
            lines.append(line)
            line = []

    return lines

def add_delimiters(image, lines, filename):
    img = cv2.imread(image)
    ih, iw, _ = img.shape
    flag = 0
    # columns = ['paxnamesector', 'sectortravel', 'sector', 'paxnametravel']
    columns = ['particulars']

    reference_pts = []

    for line in lines:
        # pprint(line)
        newline = ''.join([line[i][0] for i in range(len(line))])
        # print(newline)
        for column in columns:
            if column in newline.lower():
                # print(newline)
                flag = 1
            if flag == 1:
                # print('helloo')
                for i in range(len(line)-1):
                    if int(line[i+1][1])-int(line[i][1]) > 55:
                        cv2.putText(img, "|", ((int(line[i][3])+int(line[i+1][1]))//2, ih-(int(line[i][2]))), cv2.FONT_HERSHEY_SIMPLEX,  
                                                        1, (0, 0, 0), 2, cv2.LINE_AA)
                        # img = cv2.line(img, ((int(line[i][3])+int(line[i+1][1]))//2, ih-(int(line[i][2]))+3), ((int(line[i][3])+int(line[i+1][1]))//2, ih-(int(line[i][4]))-3), (0,0,0), 2)
                        # img = cv2.line(img, ((int(line[i][3])+int(line[i+1][1]))//2, ih-(int(line[i][2]))+3), ((int(line[i][3])+int(line[i+1][1]))//2, ih-(int(line[i][4]))-3), (0,0,0), 2)
                        # cv2.imwrite('new.jpg', img)
                        # reference_pts.append(line[i][1])
                        # print(filename)
                        cv2.imwrite('delimiters_electricity\\'+filename+'.jpg', img)
                        
    return img

def convert_text_to_separate_lines(output_file):
    '''
    Method to store text into lists of lines where each list has
    strings that are segregated by the delimiters.
    '''

    text = ''
    with open(output_file+'.txt', 'r', encoding='utf-8') as fp:
        text = fp.read()

    text = text.splitlines()

    spaces = ['', ' ']
    text = [st for st in text if st not in spaces]

    # pprint(text)

    i = 0
    for line in text:
        if re.search('.*pax.*sector.*', line.lower()):
            text = text[i:]
            break
        i += 1

    for j in range(len(text)):
        if '_' in text[j]:
            text[j] = text[j].replace('_', ' ')
        text[j] = text[j].split(' | ')
    
    return text

def get_table(lines):
    '''
    Method to get table coordinates where the rows are divided into separate lists as per the strings' respective columns
    '''
    i = 0
    for line in lines:
        i += 1
        newline = ''.join([line[i][0] for i in range(len(line))])
        # print(newline)
        if re.search('.*pax.*sector.*', newline.lower()):
            column_names = newline.lower().split('|')
            break
            # print(i)

    table = []
    table = lines[i-1:]
    extras = ['~', '_']

    # pprint(table)
    # k = 0

    for i in range(len(table)):
        table[i] = [ch for ch in table[i] if ch[0] not in extras]

    # pprint(table)

    cols = []
    col = []

    for t in table:
        row = []
        for ch in t:
            col.append(ch)
            if ch[0] == '|' or ch == t[-1]:
                if ch[0] == '|':
                    col.remove(ch)
                row.append(col)
                col = []
        cols.append(row)
    
    # pprint(cols)
    
    return cols

def merge_rows(column_coords, columns, line_items, line_item_coords, index):
    row = {}
    
    for col in column_coords:
        flag = 0
        for word in line_item_coords:
            if abs(int(col[0][1])-int(word[0][1])) <= 5:
                # print(columns[column_coords.index(col)]+'---->'+line_items[index][line_item_coords.index(word)])
                # print(line_items[index][line_item_coords.index(word)])
                # data.append({columns[column_coords.index(col)]: line_items[index][line_item_coords.index(word)]})
                flag = 1
                row[columns[column_coords.index(col)]] = line_items[index][line_item_coords.index(word)]
                # break
            elif abs(int(col[-1][1])-int(word[-1][1])) <= 20:
                # print(line_items[index][line_item_coords.index(word)])
                # print(columns[column_coords.index(col)]+'---->'+line_items[index][line_item_coords.index(word)])
                # data.append({columns[column_coords.index(col)]: line_items[index][line_item_coords.index(word)]})
                flag = 1
                row[columns[column_coords.index(col)]] = line_items[index][line_item_coords.index(word)]
            elif flag == 0:
                row[columns[column_coords.index(col)]] = None
                
    data.append(row)
    return data

def split_filename(image):
    filename = os.path.split(image)[-1].split('.jpg')[0]
    return filename

In [6]:
from prettyprinter import pprint
import glob

if __name__ == '__main__':
    
    # image = 'C:\\air_ticket\\images2\\74_Yatra_AAAIN233675535_10.09.2022_page0.jpg'
    for image in glob.glob('electricity_images\\*.jpg'):
        filename = split_filename(image)
        print(filename)

        # image = preprocess(image)
        # get_parsed_output(image, 'testoutput')

        output_path = 'box_files_electricity\\{}'.format(filename)
        get_box_file(image, output_path)

        box_file = 'box_files_electricity\\{}.box'.format(filename)
        lines = store_separate_lines(box_file)
        # pprint(lines[:3])
        # print('---------------')
        
        image = add_delimiters(image, lines, filename)
        # print(image[0])
        # print('-----------------------------------')

        # image = cv2.imread(image)
        # cv2.imwrite('delimiters\\'+filename+'.jpg', image)

        delimiter_image = 'delimiters_electricity\\'+filename+'_delimiters'+'.jpg'

        output_file = 'delimiters_electricity\\{}'.format(filename)
        get_parsed_output(delimiter_image, output_file)

        # lines = store_separate_lines(box_file)

        # text = convert_text_to_separate_lines(output_file)

        # cols = get_table(lines)

        # column_coords = cols[0]
        # line_items_coords = cols[1:]

        # columns = text[0]
        # line_items = text[1:]

        # for i in range(len(line_items_coords)):
        #     if len(columns) != len(line_items[i]):
        #         merge_rows(column_coords, columns, line_items, line_items_coords[i], i)
        #         # print('-------------------------')

    # pprint(data)  


Chennai_Office_EB_Apr_22_page0
Chennai_Office_EB_Apr_22_page1
Chennai_Office_EB_Apr_22_page2
Chennai_Office_EB_Apr_22_page3
Chennai_Office_EB_Apr_22_page4
Chennai_Office_EB_Aug_22_1_page0
Chennai_Office_EB_Aug_22_1_page1
Chennai_Office_EB_Aug_22_1_page2
Chennai_Office_EB_Aug_22_page0
Chennai_Office_EB_Aug_22_page1
Chennai_Office_EB_Aug_22_page2
Chennai_Office_EB_Aug_22_page3
Chennai_Office_EB_Dec_22_1_page0
Chennai_Office_EB_Dec_22_1_page1
Chennai_Office_EB_Dec_22_1_page2
Chennai_Office_EB_Dec_22_1_page3
Chennai_Office_EB_Dec_22_page0
Chennai_Office_EB_Dec_22_page1
Chennai_Office_EB_Dec_22_page2
Chennai_Office_EB_Dec_22_page3
Chennai_Office_EB_Feb_22_page0
Chennai_Office_EB_Feb_22_page1
Chennai_Office_EB_Feb_22_page2
Chennai_Office_EB_Feb_22_page3
Chennai_Office_EB_Jan_22_page0
Chennai_Office_EB_Jan_22_page1
Chennai_Office_EB_Jan_22_page2
Chennai_Office_EB_Jan_22_page3
Chennai_Office_EB_Jul_22_page0
Chennai_Office_EB_Jul_22_page1
Chennai_Office_EB_Jul_22_page2
Chennai_Office_EB_Jul_22_

In [ ]:
command = 'tesseract ensemble_delimiters_supplier\\Dec_2022_page0_convert_to_grayscale.jpg stdout -l eng --psm 4'
print(os.popen(command).read())

In [7]:
for image in glob.glob('ensemble_delimiters_supplier\\*'):
    filename = os.path.split(image)[-1].split('.jpg')[0]
    # print(filename)

    output_file = 'delimiters_supplier\\{}'.format(filename)
    command = 'tesseract {} {} -l eng --psm 4'.format(image, output_file)
    os.system(command)


In [2]:
import glob
print(len(glob.glob('preprocessed_ensemble\\*.jpg')))

1510


In [4]:
from prettyprinter import pprint
import glob

if __name__ == '__main__':
    
    # image = 'C:\\air_ticket\\images2\\74_Yatra_AAAIN233675535_10.09.2022_page0.jpg'
    for image in glob.glob('C:\\air_ticket\\preprocessed_ensemble\\*.jpg')[:200]:
        filename = split_filename(image)

        image = preprocess(image)
        # get_parsed_output(image, 'testoutput')

        output_path = 'boxfiles_ensemble\\{}'.format(filename)
        get_box_file(image, output_path)

        box_file = 'boxfiles_ensemble\\{}.box'.format(filename)
        lines = store_separate_lines(box_file)
        # pprint(lines[:3])
        # print('---------------')
        
        image = add_delimiters(image, lines, filename)
        # print(image[0])
        # print('-----------------------------------')

        # image = cv2.imread(image)
        # cv2.imwrite('delimiters\\'+filename+'.jpg', image)

        delimiter_image = 'ensemble_delimiters\\'+filename+'.jpg'

        output_file = 'ensemble_delimiters\\{}'.format(filename)
        get_parsed_output(delimiter_image, output_file)

        # lines = store_separate_lines(box_file)

        # text = convert_text_to_separate_lines(output_file)

        # cols = get_table(lines)

        # column_coords = cols[0]
        # line_items_coords = cols[1:]

        # columns = text[0]
        # line_items = text[1:]

        # for i in range(len(line_items_coords)):
        #     if len(columns) != len(line_items[i]):
        #         merge_rows(column_coords, columns, line_items, line_items_coords[i], i)
        #         # print('-------------------------')

    # pprint(data)  


In [9]:
import dateparser
from datetime import datetime
import glob
from prettyprinter import pprint
import pandas as pd
import cv2
import os
import re
import airportsdata

def is_date(string):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    # formats = ['%d %b']
    # try: 
    #     print(dateparser.parse(string, date_formats = formats))
    #     return True

    # except ValueError:
    #     return False

    formats = ["%d %b %Y", "%d%b %Y", "%d%b%Y"]
 
    # checking if format matches the date
    res = False
    
    # using try-except to check for truth value
    for format in formats:
        try:
            res = bool(datetime.strptime(string, format))
            return res
        except ValueError:
            res = False
    return res

# def contains_date(string):
#     date_formats = ['%d%b %Y', '%d %b %Y']
#     for format in date_formats:
#         try:
#             datetime.strptime(re.search(format, string).group(), format)
#             return True
#         except (AttributeError, ValueError):
#             pass
#             # print(e)
#     return False

def darken_text(image, k, filter):
    img = cv2.imread(image)
    if filter == 'gray':
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if filter == 'rgb':
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # k = 3
    result = 255*(img/255)**k
    cv2.imwrite('result.jpg', result)
    return 'result.jpg'

def extract_name(text):
    name = ''
    flag = 0
    for i in range(len(text)-1):
        if 'name' in text[i].lower() and 'sector' not in text[i].lower():
            flag = 1
            if ':' in text[i]:
                name =  text[i].strip().split('Name:')[-1].strip()
                break
            if '-' in text[i]:
                name = text[i].strip().split('Name - ')[-1].strip()
                break
    if flag == 0:
        for i in range(len(text)):
            if 'pax name' in text[i].lower():
                if 'mr' in text[i-1].lower():
                    name = text[i-1].strip()
                    break

    if 'adt' in name.lower():
        ind = name.lower().index('adt')
        name = name[:ind+3]
        if ')' not in name:
            if '.' in name[-1]:
                name = name.replace('.', ')')
            else:
                name += ')'
        else:
            i = name.index(')')
            name = name[:i+1]
    
    if name[-2:].lower() == 'ad':
        name += 'T)'

    extras = ['Name - ', 'Names', 'name.']

    for extra in extras:
        if extra in name:
            name = name.strip(extra)
    
    if '|' in name:
        if name.index('|') == 0:
            name = name.replace('|', '')
        else:
            name = name.replace('|', 'I')

    return name

def remove_exc(sc):
    if len(sc) == 3 and '!' in sc:
        sc = sc.replace('!', 'I')
    elif len(sc) > 3 and '!' in sc:
        sc = sc.replace('!', '')
    if '(' in sc:
        i = sc.index('(')
        sc = sc[:i]
    if '.' in sc:
        sc = sc.replace('.', '')
    if '=' in sc:
        sc = sc.replace('=', '')
    return sc
    

def extract_source_city(text):
    # print('source city-------------')
    # pprint(text)
    source_cities = []
    destination_cities = []
    flag1 = 0
    debarred = ['spicejet', 'india', 'sg', 'al', 'airline', 'airlines', 'i5', 'asia', '15', 'jet', 'u2', 'airway', '6e', '6684)', '‘qr']
    for i in range(len(text)-1):
        if 'sector' in text[i].lower() or 'pax' in text[i].lower() or 'travel' in text[i].lower():
            flag1 = 1
            sc = ''
            dc = ''
            flag2 = 0
            for j in range(i+1, len(text)):
                # print(text[j])
                words = text[j].split()
                # print(words)
                for word in words:
                    # print(word)
                    try:
                        if '-' in word:
                            ind = word.find(next(filter(str.isalpha, word)))
                            word = word[ind:]
                            # print(word)
                    except:
                        pass
                        
                    if 'airlines' in word.split('-')[0].lower():
                        # print('AIRLINESSSS')
                        continue
                    if 'indigo' in word.split('-')[0].lower() or 'charge' in word.split('-')[0].lower():
                        # print('INDIGOOOOO')
                        break
                    #  and not any(chr.isdigit() for chr in word)
                    if '-' in word and len(word.split('-')[0]) >= 3 and '(' not in word.split('-')[0] and '{' not in word:
                        
                        if '(' in word:
                            ind = word.index('(')
                            word = word[:ind]
                        ch_ind = word.find(next(filter(str.isalpha, word)))
                        # print(ch_ind)
                        word = word[ch_ind:]
                        
                        sc = word.split('-')[0]
                        sc = remove_exc(sc)
                        if sc.lower() not in debarred and len(sc) == 3:
                            source_cities.append(sc)

                        dc = word.split('-')[1]
                        dc = remove_exc(dc)
                        if dc.lower() not in debarred and len(dc) == 3:
                            destination_cities.append(dc)
                        continue

        if flag1 == 1:
            break
    
    return source_cities, destination_cities

def extract_start_date(text):
    start_dates = []
    flag1 = 0
    months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    for i in range(len(text)-1):
        if 'travel date' in text[i].lower():
            flag1 = 1
            for j in range(i+1, len(text)):
                words = text[j].split('|')
                for word in words:
                    # print(word+'---->'+str(is_date(word.strip())))
                    if is_date(word.strip(" ,")):
                        date = word.strip(" ,")
                        # if len(word[i]) == 5:
                        #     date = word[i][:2]+" "+word[i][2:]+" "+word[i+1]
                        # elif len(word[i]) == 7:
                        #     date = word[i-1]+" "+word[i][:3]+" "+word[i][3:]
                        # elif len(word[i]) == 9:
                        #     date = word[i][:2]+" "+word[i][2:5]+" "+word[i][5:]
                        # elif len(word[i]) == 3:
                        #     date = word[i-1]+" "+word[i]+" "+word[i+1]
                        # if date[:2].isnumeric():
                        #     if '.' in date:
                        #         date = date.replace('.', '')
                        #     start_dates.append(date)
                        start_dates.append(date)
                        continue
                    else:
                        flag2 = 0
                        date = ''
                        debarred = ['remarks:', 'separate', 'connaught']
                        word = word.split()
                        for i in range(len(word)):
                            for month in months:
                                if month in word[i].lower() and word[i].lower() not in debarred:
                                    flag2 = 1
                                    ind = word[i].lower().index(month)
                                    # print(word[i])
                                    word[i] = word[i].strip('.')
                                    try:
                                        if not word[i][ind-1].isalpha() or not word[i-1].isalpha():
                                            if len(word[i]) == 5:
                                                date = word[i][:2]+" "+word[i][2:]+" "+word[i+1]
                                            elif len(word[i]) == 7:
                                                date = word[i-1]+" "+word[i][:3]+" "+word[i][3:]
                                            elif len(word[i]) == 9:
                                                date = word[i][:2]+" "+word[i][2:5]+" "+word[i][5:]
                                            elif len(word[i]) == 3:
                                                date = word[i-1]+" "+word[i]+" "+word[i+1]
                                            if date[:2].isnumeric():
                                                if '.' in date:
                                                    date = date.replace('.', '')
                                                start_dates.append(date)
                                                break
                                    except:
                                        pass
                            if flag2 == 1:
                                continue
        if flag1 == 1:
            break

    return start_dates

ticket_details = {}
names = []
source_cities = []
destination_cities = []
start_dates = []
end_dates = []
# C:\air_ticket\images2\74_Yatra_AAAIN233675535_10.09.2022_page0.jpg
for file in glob.glob('C:\\air_ticket\\delimiters2\\*.txt'):
    with open(file, 'r', encoding='utf-8') as fp:
        text = fp.read()
    text = text.splitlines()
    text = [line for line in text if not line.isspace() and len(line) > 0]
    # pprint(text)
    filename = os.path.split(file)[-1].split('.txt')[0]

    # pprint(text)
    print(filename)
    # print()

    # name = extract_name(text)
    # print(name)
    # names = []
    # source_cities = []
    # destination_cities = []
    # start_dates = []
    # end_dates = []

    name = extract_name(text)
    if len(name) == 0:
        ## call darken text function and call extract name function again
        img_file = 'C:\\air_ticket\\images2\\{}.jpg'.format(filename)
        dark_image = darken_text(img_file, 3, 'gray')
        command = 'tesseract {} text_output -l eng --psm 4'.format(dark_image)
        os.system(command)
        with open('text_output.txt', 'r', encoding='utf-8') as fp:
            text_for_name = fp.read()
        text_for_name = text_for_name.splitlines()
        text_for_name = [line for line in text_for_name if not line.isspace() and len(line) > 0]
        # print('NAME---------------------')
        # pprint(text)
        name = extract_name(text_for_name)

    if len(name) == 0:
        img_file = 'C:\\air_ticket\\delimiters2\\{}.jpg'.format(filename)
        dark_image = darken_text(img_file, 3, 'gray')
        command = 'tesseract {} text_output -l eng --psm 4'.format(dark_image)
        os.system(command)
        with open('text_output.txt', 'r', encoding='utf-8') as fp:
            text_for_name2 = fp.read()
        text_for_name2 = text_for_name2.splitlines()
        text_for_name2 = [line for line in text_for_name2 if not line.isspace() and len(line) > 0]
        # print('NAME---------------------')
        # pprint(text)
        name = extract_name(text_for_name2)

    if len(name) == 0:
        img_file = 'C:\\air_ticket\\delimiters2\\{}.jpg'.format(filename)
        dark_image = darken_text(img_file, 4, 'gray')
        command = 'tesseract {} text_output -l eng --psm 4'.format(dark_image)
        os.system(command)
        with open('text_output.txt', 'r', encoding='utf-8') as fp:
            text_for_name3 = fp.read()
        text_for_name3 = text_for_name3.splitlines()
        text_for_name3 = [line for line in text_for_name3 if not line.isspace() and len(line) > 0]
        # print('NAME---------------------')
        # pprint(text)
        name = extract_name(text_for_name3)
    
    if len(name) == 0:
        img_file = 'C:\\air_ticket\\images2\\{}.jpg'.format(filename)
        dark_image = darken_text(img_file, 5, 'rgb')
        command = 'tesseract {} text_output -l eng --psm 4'.format(dark_image)
        os.system(command)
        with open('text_output.txt', 'r', encoding='utf-8') as fp:
            text_for_name4 = fp.read()
        text_for_name4 = text_for_name4.splitlines()
        text_for_name4 = [line for line in text_for_name4 if not line.isspace() and len(line) > 0]
        # print('NAME---------------------')
        # pprint(text)
        name = extract_name(text_for_name4)

    if len(name) == 0:
        img_file = 'C:\\air_ticket\\delimiters2\\{}.jpg'.format(filename)
        dark_image = darken_text(img_file, 6, 'gray')
        command = 'tesseract {} text_output -l eng --psm 4'.format(dark_image)
        os.system(command)
        with open('text_output.txt', 'r', encoding='utf-8') as fp:
            text_for_name5 = fp.read()
        text_for_name5 = text_for_name5.splitlines()
        text_for_name5 = [line for line in text_for_name5 if not line.isspace() and len(line) > 0]
        # print('NAME---------------------')
        # pprint(text)
        name = extract_name(text_for_name5)

    print('Name ----> ', name)
    source_city, destination_city = extract_source_city(text)
    print('Source City ----> ', source_city)
    print('Destination City ----> ', destination_city)
    start_date = extract_start_date(text)
    end_date = extract_start_date(text)
    print('Start Date ----> ', start_date)
    print('End Date ----> ', end_date)
    # if name is not None:
        # names.append(name*len(start_dates))
    # print(names)
    
    text_file = 'textfiles5\\{}.txt'.format(filename)
    with open(text_file, 'w', encoding='utf-8') as fp:
        if not name.strip():
            name = 'None'
        fp.write(name+"\n")
        fp.write(str(source_city)+"\n") 
        fp.write(str(destination_city)+"\n") 
        fp.write(str(start_date)+"\n")
        fp.write(str(end_date)+"\n")
        fp.write(str(filename)+"\n") 
    
    print('---------------------------------------------------------')


    l = len(destination_city)

    for i in range(len(source_city)):
        names.append(name)
    
    for i in range(len(source_city)):
        source_cities.append(source_city[i])
    
    for i in range(len(destination_city)):
        destination_cities.append(destination_city[i])
    
    for i in range(len(start_date)):
        start_dates.append(start_date[i])
        
    for i in range(len(end_date)):
        end_dates.append(end_date[i])

    # ticket_details['Page'] = filename
    ticket_details['Name'] = names
    ticket_details['Source City'] = source_cities
    ticket_details['Destination City'] = destination_cities
    ticket_details['Start Date'] = start_dates
    ticket_details['End Date'] = end_dates

    

# ticket_details['Name'] = names
# ticket_details['Start Date'] = start_dates
# ticket_details['End Date'] = end_dates
# ticket_details['Source City'] = source_cities
# ticket_details['Destination City'] = destination_cities

    # if len(start_date) != len(source_city):
    #     print(filename)
    #     print(source_city)
    #     print(start_date)

# pprint(ticket_details)
# print(len(ticket_details['Name']))
# print(len(ticket_details['Source City']))
# print(len(ticket_details['Destination City']))
# pprint(ticket_details['Start Date'])
# print(len(ticket_details['End Date']))

# keys = ['Name', 'Source City', 'Destination City']
# # print(len(keys))

# df = pd.DataFrame(ticket_details)
# # df.index = keys
# df.to_csv('ticket_details.csv')
# print(df)




113_Yatra_AAAIN233676709_27.09.2022_page0
Name ---->  MR Nagaraju Kukkanti (ADT)
Source City ---->  ['HYD']
Destination City ---->  ['GOI']
Start Date ---->  ['03 Nov 2022']
End Date ---->  ['03 Nov 2022']
---------------------------------------------------------
114_Yatra__AAAIN233676710_27.09.2022_page0
Name ---->  MR Saran Kakollu (ADT)
Source City ---->  ['HYD']
Destination City ---->  ['GOI']
Start Date ---->  ['03 Nov 2022']
End Date ---->  ['03 Nov 2022']
---------------------------------------------------------
118_Yatra_AAAIN233676717_27.09.2022_page0
Name ---->  MR KarJakar Reddy Yerragunta (ADT)
Source City ---->  ['HYD']
Destination City ---->  ['GDI']
Start Date ---->  ['03Nov 2022']
End Date ---->  ['03Nov 2022']
---------------------------------------------------------
119_Yatra_AAAIN233676781_28.09.2022_page0
Name ---->  MR LAKHSMI NARAYANA RAJU. GINUGOORTHI (ADT)
Source City ---->  ['CPH']
Destination City ---->  ['AMS']
Start Date ---->  ['15 Oct 2022']
End Date ---->

In [90]:
count = 0
for image in glob.glob('C:\\air_ticket\\delimiters\\*.jpg'):
    count += 1

print(count)

224


In [11]:
import cv2

image = 'C:\\air_ticket\\images2\\114_Yatra__AAAIN233676710_27.09.2022_page0.jpg'
img = cv2.imread(image)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imwrite('sample.jpg', img)

command = 'tesseract {} sampleoutput -l eng --psm 4'.format('sample.jpg')
# print(os.popen(command).read())
os.system(command)

0

In [57]:
with open("C:\\air_ticket\\delimiters2\\Yatra_AAAIN223667591_17.03.2022_page0.txt") as fp:
    print(fp.read().splitlines())

['I', 'prt eee', 'r TAH 5 mene', '‘ ah Tiree ; GSTIN: 36AAACA0313P2ZV', '', 'for Business', 'PAN Card No. : AAACA0313P', 'Yatra for Business Private Limited', '', 'Corporate Identity No. :', '', '301, Dev Dhanuka Prestige, Plot No -8, U72900DL1962PTC003735', '', 'Road No -12, Banjara Hills,', '', 'HSN - 998551 |', 'HYDERABAD-500034, TELANGANA,', '', 'INDIA', '', 'Entity', 'Address', 'State Code 136 | Place of Supply : Telangana', 'GSTIN : 36AACCC8032H1ZT PAN Number : AACCC8032H', 'BookingDetails:', '', 'Booking Ref. : CT5281505 Booking Date : 117 Mar 2022', 'Invoice No. : : AAAIN223667591 Invoice Date : > 17 MAR 2022', 'Booking Type : Official', '', 'Trip Id : TA1703220031486 Irn Number', '', '9b0601616e8094d7ff0065b49b9ea', '97aaa4e4945751332ae85003faad', '8f40773', '', 'Pax Name | Sector Travel Date | Ticket Number PNR No.', 'Mr SHIVA PRASAD | HYD-DEL 22 Mar 2022 | 6997156839 JZTPQ7', '', 'Breakup: BaseFare7470.00,IN332.00,P2236.00,YR130.00)Air Fare (collected on behalf of airline)* 